In [1]:
import pandas as pd
import urllib3
import json
import requests
from io import StringIO

In [2]:
"""
PART0:
Read csv data from pr.data.0.Current from url
"""

url = "https://download.bls.gov/pub/time.series/pr/pr.data.0.Current"
s = requests.get(url).content
time_series_csv_df = pd.read_csv((StringIO(s.decode('utf-8'))), sep='\t')

time_series_csv_df.head()

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [3]:
"""
PART0:
Read json data from the given API 
"""

http = urllib3.PoolManager()
r = http.request('GET', 'https://datausa.io/api/data?drilldowns=Nation&measures=Population')
#print (r.data.decode('UTF-8'))

population_json = json.loads(r.data.decode('UTF-8'))
temp_data =  population_json["data"]
population_df = pd.DataFrame(temp_data)

population_df.head()

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2019,2019,328239523,united-states
1,01000US,United States,2018,2018,327167439,united-states
2,01000US,United States,2017,2017,325719178,united-states
3,01000US,United States,2016,2016,323127515,united-states
4,01000US,United States,2015,2015,321418821,united-states


In [5]:
"""
PART1:
Using the dataframe from the population data API (Part 2), generate the mean and
the standard deviation of the US population across the years [2013, 2018] inclusive.

"""
std_json_data = population_df['Population'].std()
mean_json_data = population_df['Population'].mean()
# print('Standard Deviation: ' + std_json_data)
# print('Mean: ' + mean_json_data)
print(std_json_data)
print(mean_json_data)


4455158.63450706
322951195.85714287


In [6]:
"""
PART 2:
Using the dataframe from the time-series (Part 1), For every series_id, find the 
best year: the year with the max/largest sum of "value" for all quarters in 
that year. Generate a report with each series id, the best year for that series,
and the summed value for that year. For example, if the table had the following values:
"""

time_series_csv_df.columns = time_series_csv_df.columns.str.strip()
time_series_csv_df_grouped = time_series_csv_df.groupby(['series_id','year']).agg({'value': 'sum'}).reset_index()
#print(time_series_csv_df_grouped.head(30)) 
result = time_series_csv_df_grouped.groupby(['series_id'], as_index=False)['value'].max().rename(columns={'value': 'value'})
final_result = time_series_csv_df_grouped.merge(result, on=['series_id', 'value'])
#print(final_result)
final_result.head()


            series_id  year  value
0   PRS30006011        1995    7.1
1   PRS30006011        1996   -0.5
2   PRS30006011        1997    4.4
3   PRS30006011        1998    4.2
4   PRS30006011        1999   -7.7
5   PRS30006011        2000   -1.3
6   PRS30006011        2001  -23.3
7   PRS30006011        2002  -36.0
8   PRS30006011        2003  -23.4
9   PRS30006011        2004   -6.8
10  PRS30006011        2005   -2.5
11  PRS30006011        2006   -2.9
12  PRS30006011        2007   -8.5
13  PRS30006011        2008  -17.9
14  PRS30006011        2009  -56.3
15  PRS30006011        2010  -13.7
16  PRS30006011        2011    6.7
17  PRS30006011        2012    9.5
18  PRS30006011        2013    3.0
19  PRS30006011        2014    6.6
20  PRS30006011        2015    6.2
21  PRS30006011        2016   -0.1
22  PRS30006011        2017    4.6
23  PRS30006011        2018    8.1
24  PRS30006011        2019    5.3
25  PRS30006011        2020  -24.7
26  PRS30006011        2021    1.0
27  PRS30006012     

,series_id,year,value
0,PRS30006011,2012,9.50
1,PRS30006012,2014,8.80
2,PRS30006013,1998,733.29
3,PRS30006021,2010,14.20
4,PRS30006022,2010,11.20


In [7]:
population_df = population_df.rename(columns={'Year': 'year'})
population_df['year']=population_df['year'].astype(int)

In [8]:
"""
PART 3:
Using both dataframes from Part 1 and Part 2, generate a report that will provide 
the value for series_id = PRS30006032 and period = Q01 and the population for 
that given year (if available in the population dataset)
"""

clubbed_df = time_series_csv_df.merge(population_df, on=['year'])
clubbed_df.drop(['footnote_codes', 'ID Nation','Nation', 'ID Year', 'Slug Nation'], axis=1, inplace=True)
print(clubbed_df.head()) 



           series_id  year period  value  Population
0  PRS30006011        2013    Q01    1.3   316128839
1  PRS30006011        2013    Q02    0.2   316128839
2  PRS30006011        2013    Q03    0.2   316128839
3  PRS30006011        2013    Q04    0.7   316128839
4  PRS30006011        2013    Q05    0.6   316128839
